## Take Home Assignment

Student ID: 

Collaborators: 919755, 1153929, 1152810, 1151277, 1151248

In [33]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt 
import seaborn as sns

from statsmodels.tsa.api import VAR

from statistics import multimode
from numpy.linalg import det, inv
from numpy.linalg import matrix_power as mp

In [34]:
def Z_matrix(y: np.array, p: int, c: int):
    """Calculate the Z-matrix for a given input

    Args:
        y (np.array): input with all the data of shape (T + p) × K
        p (int): lags
        c (int): intercept yes=1, no=0

    Returns:
        (np.array): Z-matrix for given input
    """

    y = y.T

    #determine matrix dimensions:
    T = y.shape[1] - p
    K = y.shape[0]

    # build Z-matrix
    if c==1:
        Z = np.ones((1, T+p), dtype=float)

    # 1b stacked lagged data
    for i in range(p):
        #add i columns of leading zeros (EDIT: empty, comp cost lower) to ktpmat
        zeros = np.zeros((K, i), dtype=float)
        zerostack = np.hstack((zeros, y[:,:(T+p-i)]))
        # vertically stack this to Z
        Z = np.vstack((Z, zerostack))

    # cutting of leading p columns and retrieving Z
    Z = Z[:, p-1:-1]

    return Z

In [35]:
def B_matrix(y: np.array, p: int, c: int):
    """Calculates the B matrix with the estimated coefficients

    Args:
        y (np.array): input with all the data of shape (T + p) × K
        p (int): lags
        c (int): intercept yes=1, no=0

    Returns:
        _type_: B = matrix with estimated coefficients; Z=Z-matrix; sigma_u=covariance matrix
    """

    # get Z-matrix from function above
    Z = Z_matrix(y, p, c)

    y = y.T # transpose y
    y = y[:,p:] # first p observations are lost as we need prior lags for estimation
    K = y.shape[0] # number of variables
    T = y.shape[1] # number of observations

    # calculate B
    B = y @ Z.T @ np.linalg.inv((Z@Z.T))

    # calculate sigma_u (covariance matrix)
    sigma_u = (1/(T-K*p-1))*(y-(B@Z))@(y-(B@Z)).T

    return B, Z, sigma_u

## Exercise 1

Exercise 1(b-d)

In [36]:
def var1sim(A1: np.array, sigma_u: np.array, T: int):
    """A function that simulates time series data from a K-dimensional VAR(1) process yt = A1 y_t−1 + u_t, 
    where the innovations ut are drawn from a multivariate normal distribution with mean zero and covariance matrix Σ_u. 
    Uses y_0 = 0 as starting value, where 0 is a K × 1 vector of zeros.
    Generates time series of length T+50 and discards the first 50 observations, 
    such that it returns a time series of total length equal to T.

    Args:
        A1 (np.array): coefficient matrix at lag 1
        sigma_u (np.array): covariance matrix Σ_u
        T (int): number of observations

    Returns:
        np.array: T x K matrix of observations on y_t
    """
    K = sigma_u.shape[0]

    # set starting values
    y_tminus1 = np.zeros((K, 1))

    P = np.linalg.cholesky(sigma_u)

    for i in range(T+50):
        # draw disturbance u_t
        u_t = P @ np.random.standard_normal(K)
        u_t = u_t.reshape(K, 1)
        #recursively calculate y_t 
        y_t = A1@y_tminus1 + u_t
        if i == 0:
            y = y_t
        else:
            y = np.hstack((y, y_t))
        y_tminus1 = y_t

    # discard first 50 observations
    y = y[:,50:]

    return y.T

In [37]:
def var_information_criteria(y_t: np.array, p_max: int):
    """A function to calculate the 4 information criteria (FPE, AIC, HQ, SC) for a VAR process with maxlag = pmax.
       Assumes model has an intercept. Returns the chosen lag order.
       
       ### Right now also has built-in as controls ###

    Args:
        y_t (np.array): K x T matrix of observations on y_t
        p_max (int): maximum number of lags included in the model

    Returns:
        int: The chosen information criterion based on the mode of the suggestions of the 4 criteria. 
             In case of a draw, the higher one is selected.
    """
    
    # arrays to store IC values
    FPE, AIC, HQ, SC = np.zeros(p_max), np.zeros(p_max), np.zeros(p_max), np.zeros(p_max)
    b_FPE, b_AIC, b_HQ, b_SC = np.zeros(p_max), np.zeros(p_max), np.zeros(p_max), np.zeros(p_max) # built-in for control

    # get T and K from the shape of the input
    K = y_t.shape[0]
    T = y_t.shape[1] - p_max # lags needed for estimation
    
    # calculate information criteria for lags 
    for m in range(1, p_max+1):
        
        # built-in test for control
        model = VAR(y_t[:, p_max-m:].T)
        
        # estimate sigma_u_tilde with MLE (2.55 - 2.56)
        Z = Z_matrix(y_t[:, p_max-m:].T,  m, c=1) # take m lags for estimation
        B_tilde = y_t[:, p_max:] @ Z.T @ inv(Z@Z.T)
        sigma_tilde_m = (1/T)*(y_t[:, p_max:] - B_tilde@Z)@(y_t[:, p_max:] - B_tilde@Z).T
        
        # calculate ICs for given lag m (2.107 - 2.114)
        det_sigma_tilde_m = det(sigma_tilde_m)
        FPE[m-1] = (((T + K*m + 1) / (T - K*m - 1)) ** K) * det_sigma_tilde_m
        AIC[m-1] = np.log(det_sigma_tilde_m) + ((2*(K**2)*m)/T)
        HQ[m-1] = np.log(det_sigma_tilde_m) + ((2*np.log(np.log(T)))/T) * (K**2)*m
        SC[m-1] = np.log(det_sigma_tilde_m) + (np.log(T)/T) * (K**2)*m
        
        # built in for control
        result = model.fit(m)
        b_FPE[m-1] = result.fpe
        b_AIC[m-1] = result.aic
        b_HQ[m-1] = result.hqic
        b_SC[m-1] = result.bic
    
    # store all in one array and select the lowest values    
    criteria_all_lags = np.array([FPE, AIC, SC, HQ])
    lag_orders_selected = np.argmin(criteria_all_lags, axis=1)+1
    infoC = np.amin(criteria_all_lags, axis=1)
    
    # built-in for control 
    b_criteria_all_lags = np.array([b_FPE, b_AIC, b_SC, b_HQ])
    b_lag_orders_selected = np.argmin(b_criteria_all_lags, axis=1)+1
    b_infoC = np.amin(criteria_all_lags, axis=1)
    
    # select the estimated lag length. If there is a "draw", go with the higher one
    p_hat = max(multimode(lag_orders_selected))
    
    return lag_orders_selected, b_lag_orders_selected

In [38]:
def hstep_forecast(y: np.array, p: int, h: int):
    """A function that computes the h-step ahead point forecasts y_T (h) and the corresponding MSE matrix Σˆ_y(h) based on a VAR(p) with intercept

    Args:
        y (np.array): K × T matrix of observations
        p (int): lag order
        h (int): forecast horizon

    Returns:
        list: h-step ahead forecasts and the corresponding MSE matrix
    """
    
    K = y.shape[0]
    T = y.shape[1]

    # retrieving estimates
    B, Z, sigma_u = B_matrix(y.T, p, c=1)

    # constructing matrices
    J1 = np.hstack((np.zeros((K, 1)), np.identity(K), np.zeros((K, K*(p-1)))))

    row0 = np.hstack((np.ones((1,1)), np.zeros((1, K*p))))
    rowz = np.hstack((np.zeros((K*(p-1), 1)), np.identity(K*(p-1)), np.zeros((K*(p-1), K))))
    B = np.vstack((row0, B, rowz))

    Zt = y[:,-p:]                       # selecting y[:,-p:] from t-p up to t
    Zt = Zt[:,::-1].T.flatten()             # reverse order horizontally, transpose and flatten.
    Zt = np.hstack((np.array([(1)]), Zt)).reshape(K*p+1, 1)   # adding one leading 1, transposing, dimension is: 1+K*T x 1
   
     # predicting y_th
    y_th = J1@mp(B, h)@Zt
    
    # calculate the corresponding MSE matrix
    sigma_hat_yh = 0
    for i in range(h):                # formula at p. 64
        PHIi = J1@mp(B, i)@J1.T 
        part_of_sum = PHIi@sigma_u@PHIi.T
        sigma_hat_yh += part_of_sum

    return y_th, sigma_hat_yh

In [39]:
#  define parameters for the function

T1 = 50
T2 = 100
T3 = 200

a11 = 0.5
A1 = np.array([[a11, 0], 
              [0.5, 0.5]])

sigma_u = np.array([[1, 0.5],
                   [0.5, 1]])

In [40]:
def monte_carlo_va1(A1:np.array, sigma_u:np.array, T:int, M=1000):
    
    p_hat_store = np.zeros(M, dtype=int)
    
    for i in range(M):
        
        # simulate timeseries
        y_t = var1sim(A1, sigma_u, T)
        
        # reverse y_t to KxT matrix
        y_t = y_t.T
        
        # apply standard information criteria (hold out the last 4 lags for forecasting)
        p_hat = var_information_criteria(y_t[:, :T-4], 8) 
        # p_hat_store[i] = p_hat
        print(p_hat)
        
        # h-step ahead forecasts for h = 1, 2, 4
        
        
    return None #p_hat_store
        
        

In [41]:
monte_carlo_va1(A1, sigma_u, T1+4)

(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([2, 4, 1, 1], dtype=int64), array([2, 4, 1, 1], dtype=int64))
(array([3, 3, 1, 3], dtype=int64), array([3, 3, 1, 3], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([2, 2, 1, 1], dtype=int64), array([2, 2, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64),

In [42]:
monte_carlo_va1(A1, sigma_u, T2+4)

(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 6, 1, 1], dtype=int64), array([1, 6, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([3, 3, 1, 1], dtype=int64), array([3, 3, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([4, 4, 1, 1], dtype=int64), array([4, 4, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64),

In [43]:
monte_carlo_va1(A1, sigma_u, T3+4)

(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([2, 2, 1, 1], dtype=int64), array([2, 2, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([2, 2, 1, 1], dtype=int64), array([2, 2, 1, 1], dtype=int64))
(array([2, 2, 1, 2], dtype=int64), array([2, 2, 1, 2], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64),

Exercise 1(f)

In [44]:
a11 = 0.95
A1 = np.array([[a11, 0], 
              [0.5, 0.5]])

In [45]:
monte_carlo_va1(A1, sigma_u, T1+4)

(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([2, 2, 1, 2], dtype=int64), array([2, 2, 1, 2], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([2, 2, 1, 2], dtype=int64), array([2, 2, 1, 2], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64),

In [46]:
monte_carlo_va1(A1, sigma_u, T2+4)

(array([5, 5, 1, 1], dtype=int64), array([5, 5, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([3, 3, 1, 1], dtype=int64), array([3, 3, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([2, 2, 1, 1], dtype=int64), array([2, 2, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64),

In [47]:
monte_carlo_va1(A1, sigma_u, T3+4)

(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([2, 2, 1, 1], dtype=int64), array([2, 2, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([1, 1, 1, 1], dtype=int64), array([1, 1, 1, 1], dtype=int64))
(array([4, 4, 1, 1], dtype=int64),

## Exercise 2

Exercise 2(a)

Exercise 2(b)

Exercise 2(c)

Exercise 2(d)

Exercise 2(e)

Exercise 2(f)

Exercise 2(h)

Exercise 2(i)

Junk

In [48]:
def var_information_criteria_given(y_t: np.array, sigma_u: np.array, p_max: int):
    """A function to calculate the 4 information criteria (FPE, AIC, HQ, SC) for a VAR process with maxlag = pmax.
       Assumes the covariance matrix Σ_u is known. Return the chosen lag order.

    Args:
        y_t (np.array): K x T matrix of observations on y_t
        sigma_u (np.array): covariance matrix Σ_u
        p_max (int): maximum number of lags included in the model

    Returns:
        int: The chosen information criterion based on the mode of the suggestions of the 4 criteria. 
             In case of a draw, the higher one is selected.
    """
    
    # arrays to store IC values
    FPE, AIC, HQ, SC = np.zeros(p_max), np.zeros(p_max), np.zeros(p_max), np.zeros(p_max)

    # get T and K from the shape of the input
    K = y_t.shape[0]
    T = y_t.shape[1]
    
    # calculate information criteria for lags 
    for m in range(1, p_max+1):
        
        # calculate ICs for given lag m (2.107 - 2.114)
        FPE[m-1] = (((T + K*m + 1) / (T - K*m - 1)) ** K) * det(sigma_u)
        AIC[m-1] = np.log(det(sigma_u)) + ((2*(K**2)*m)/T)
        HQ[m-1] = np.log(det(sigma_u)) + ((2*np.log(np.log(T)))/T) * (K**2)*m
        SC[m-1] = np.log(det(sigma_u)) + (np.log(T)/T) * (K**2)*m
    
    # store all in one array and select the lowest values    
    criteria_all_lags = np.array([FPE, AIC, SC, HQ])
    lag_orders_selected = np.argmin(criteria_all_lags, axis=1)+1
    
    # select the estimated lag length. If there is a "draw", go with the higher one
    p_hat = max(multimode(lag_orders_selected))
    
    return p_hat